In [1]:
import pandas as pd
import requests
from datetime import datetime

pd.set_option('display.max_columns', None)

In [2]:
config = pd.read_csv('sandbox.config')
token = config.prod_token[0]

In [3]:
endpoint = 'https://api.tradier.com'
symbol = 'AAPL'

In [28]:
path = 'user/profile'
response = requests.get(f'{endpoint}{path}',
    params = {},
    headers = {'Authorization': f'Bearer {token}', 
               'Accept': 'application/json'}
)

In [4]:
path = '/v1/markets/options/expirations'
response = requests.get(f'{endpoint}{path}',
    params={'symbol': f'{symbol}', 
            'includeAllRoots': 'true'},
    headers={'Authorization': f'Bearer {token}', 
             'Accept': 'application/json'}
)

In [5]:
json_response = response.json()


In [59]:
dates = []

for i in range(1, len(json_response['expirations']['expiration'])):
    date = json_response['expirations']['expiration'][i]['date']
    dates.append(date)

In [8]:
dates = json_response['expirations']['date']

In [74]:
path = '/v1/markets/options/chains'

all_chains = pd.DataFrame()

for date in dates:
    response = requests.get(f'{endpoint}{path}',
        params={'symbol': f'{symbol}', 
                'expiration': f'{dates[0]}', 
                'greeks': 'true'},
        headers={'Authorization': f'Bearer {token}', 
                 'Accept': 'application/json'}
    )

    json_response = response.json()
    
    tmp = pd.json_normalize(json_response['options']['option'])
    
    all_chains = pd.concat([all_chains, tmp], axis = 0, ignore_index = True)

In [92]:
all_chains.loc[all_chains['option_type'] == 'put']

,symbol,description,exch,type,last,change,volume,open,high,low,close,bid,ask,underlying,strike,change_percentage,average_volume,last_volume,trade_date,prevclose,week_52_high,week_52_low,bidsize,bidexch,bid_date,asksize,askexch,ask_date,open_interest,contract_size,expiration_date,expiration_type,option_type,root_symbol,greeks.delta,greeks.gamma,greeks.theta,greeks.vega,greeks.rho,greeks.phi,greeks.bid_iv,greeks.mid_iv,greeks.ask_iv,greeks.smv_vol,greeks.updated_at
0,AAPL221202P00050000,AAPL Dec 2 2022 $50.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,None,0.00,0.01,AAPL,50.0,NaN,0,0,0,NaN,0.0,0.0,0,J,1669142106000,1115,W,1669143266000,0,100,2022-12-02,weeklys,put,AAPL,0.0,-9.070896e-16,-6.261837e-03,0.00002,1.385604e-02,-4.138791e-02,0.0,2.173630,2.173630,0.519,2022-11-22 17:58:51
2,AAPL221202P00055000,AAPL Dec 2 2022 $55.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,None,0.00,0.01,AAPL,55.0,NaN,0,0,0,NaN,0.0,0.0,0,J,1669142106000,1120,W,1669143266000,0,100,2022-12-02,weeklys,put,AAPL,0.0,-9.070896e-16,-6.888021e-03,0.00002,1.524165e-02,-4.138791e-02,0.0,1.991896,1.991896,0.519,2022-11-22 17:58:51
4,AAPL221202P00060000,AAPL Dec 2 2022 $60.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,None,0.00,0.01,AAPL,60.0,NaN,0,0,0,NaN,0.0,0.0,0,J,1669142106000,1117,W,1669143266000,0,100,2022-12-02,weeklys,put,AAPL,0.0,-9.890787e-16,-7.514205e-03,0.00002,1.662725e-02,-4.138791e-02,0.0,1.825145,1.825145,0.519,2022-11-22 17:58:51
6,AAPL221202P00065000,AAPL Dec 2 2022 $65.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,None,0.00,0.01,AAPL,65.0,NaN,0,0,0,NaN,0.0,0.0,0,J,1669142106000,1118,W,1669143266000,0,100,2022-12-02,weeklys,put,AAPL,0.0,-9.070896e-16,-8.140389e-03,0.00002,1.801286e-02,-4.138791e-02,0.0,1.671047,1.671047,0.519,2022-11-22 17:58:51
8,AAPL221202P00070000,AAPL Dec 2 2022 $70.00 Put,Z,option,0.01,0.0,5,0.01,0.01,0.01,None,0.00,0.01,AAPL,70.0,0.0,0,5,1669135639349,0.01,0.0,0.0,0,J,1669142106000,1083,W,1669143266000,1372,100,2022-12-02,weeklys,put,AAPL,0.0,-2.709586e-15,-8.766572e-03,0.00002,1.939846e-02,-4.138791e-02,0.0,1.528599,1.528599,0.519,2022-11-22 17:58:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336,AAPL221202P00230000,AAPL Dec 2 2022 $230.00 Put,Z,option,92.20,0.0,0,NaN,NaN,NaN,None,79.85,80.20,AAPL,230.0,0.0,0,1,1668006568977,92.20,0.0,0.0,90,B,1669143254000,38,T,1669143255000,0,100,2022-12-02,weeklys,put,AAPL,-1.0,1.409663e-28,-3.091513e-28,0.00002,3.948756e-30,-3.966738e-30,0.0,1.229679,1.229679,0.333,2022-11-22 17:58:51
2338,AAPL221202P00235000,AAPL Dec 2 2022 $235.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,None,84.90,85.20,AAPL,235.0,NaN,0,0,0,NaN,0.0,0.0,39,T,1669143150000,11,Z,1669143205000,0,100,2022-12-02,weeklys,put,AAPL,-1.0,0.000000e+00,0.000000e+00,0.00002,0.000000e+00,0.000000e+00,0.0,1.280686,1.280686,0.333,2022-11-22 17:58:51
2340,AAPL221202P00240000,AAPL Dec 2 2022 $240.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,None,89.85,90.20,AAPL,240.0,NaN,0,0,0,NaN,0.0,0.0,90,B,1669143250000,37,T,1669143250000,0,100,2022-12-02,weeklys,put,AAPL,-1.0,0.000000e+00,0.000000e+00,0.00002,0.000000e+00,0.000000e+00,0.0,1.330615,1.330615,0.333,2022-11-22 17:58:51
2342,AAPL221202P00245000,AAPL Dec 2 2022 $245.00 Put,Z,option,107.25,0.0,0,NaN,NaN,NaN,None,94.85,95.20,AAPL,245.0,0.0,0,1,1668006568945,107.25,0.0,0.0,90,B,1669143269000,39,T,1669143269000,0,100,2022-12-02,weeklys,put,AAPL,-1.0,0.000000e+00,0.000000e+00,0.00002,0.000000e+00,0.000000e+00,0.0,1.379510,1.379510,0.333,2022-11-22 17:58:51


In [81]:
path = '/v1/markets/quotes'

response = requests.get(f'{endpoint}{path}',
    params={'symbols': f'{symbol}', 
            'greeks': 'false'},
    headers={'Authorization': f'Bearer {token}', 
             'Accept': 'application/json'}
)

json_response = response.json()
price = json_response['quotes']['quote']['last']

In [82]:
price

149.88

In [87]:
path = '/v1/markets/history'

response = requests.get(f'{endpoint}{path}',
    params={'symbol':f'{symbol}', 
            'interval': 'daily', 
            'start': '2000-01-01', 
            'end': f'{datetime.today().date().isoformat()}'},
    headers={'Authorization': f'Bearer {token}', 
             'Accept': 'application/json'}
)
json_response = response.json()

In [91]:
pd.json_normalize(json_response['history']['day'])

,date,open,high,low,close,volume
0,2000-01-03,0.936384,1.004464,0.907924,0.999442,535796800
1,2000-01-04,0.966518,0.987723,0.903460,0.915179,512377600
2,2000-01-05,0.926339,0.987165,0.919643,0.928571,778321600
3,2000-01-06,0.947545,0.955357,0.848214,0.848214,767972800
4,2000-01-07,0.861607,0.901786,0.852679,0.888393,460734400
...,...,...,...,...,...,...
5756,2022-11-16,149.130000,149.870000,147.290000,148.790000,64218266
5757,2022-11-17,146.430000,151.480000,146.150000,150.720000,80389400
5758,2022-11-18,152.305000,152.700000,149.970000,151.290000,74829573
5759,2022-11-21,150.160000,150.370000,147.715000,148.010000,58724070


In [11]:
path = '/v1/markets/options/chains'

response = requests.get(f'{endpoint}{path}',
    params={'symbol': f'{symbol}', 
            'expiration': f'{dates[0]}', 
            'greeks': 'true'},
    headers={'Authorization': f'Bearer {token}', 
             'Accept': 'application/json'}
)

json_response = response.json()

json_response

{'options': {'option': [{'symbol': 'AAPL221125P00070000',
    'description': 'AAPL Nov 25 2022 $70.00 Put',
    'exch': 'Z',
    'type': 'option',
    'last': 0.01,
    'change': 0.0,
    'volume': 3,
    'open': 0.01,
    'high': 0.01,
    'low': 0.01,
    'close': None,
    'bid': 0.0,
    'ask': 0.01,
    'underlying': 'AAPL',
    'strike': 70.0,
    'greeks': {'delta': 8.2e-15,
     'gamma': 9.116465401174264e-14,
     'theta': -0.007651206621923369,
     'vega': 1.9999995844647873e-05,
     'rho': 0.005922361832082895,
     'phi': -0.012687449242321236,
     'bid_iv': 0.0,
     'mid_iv': 0.0,
     'ask_iv': 0.0,
     'smv_vol': 0.392,
     'updated_at': '2022-11-22 18:58:52'},
    'change_percentage': 0.0,
    'average_volume': 0,
    'last_volume': 1,
    'trade_date': 1669130216062,
    'prevclose': 0.01,
    'week_52_high': 0.0,
    'week_52_low': 0.0,
    'bidsize': 0,
    'bidexch': 'J',
    'bid_date': 1669146215000,
    'asksize': 1129,
    'askexch': 'W',
    'ask_date': 1

In [12]:
start = '2000-01-01'
end = datetime.today().date().isoformat()
endpoint = 'https://api.tradier.com'
path = '/v1/markets/history'

In [13]:
response = requests.get(f'{endpoint}{path}',
        params = {'symbol':f'{symbol}', 
                  'interval': 'daily', 
                  'start': '2000-01-01', 
                  'end': f'{datetime.today().date().isoformat()}'},
        headers = {'Authorization': f'Bearer {token}', 
                   'Accept': 'application/json'})
json_response = response.json()

In [14]:
hist_price = pd.json_normalize(json_response['history']['day'])

In [19]:
hist_price

,date,open,high,low,close,volume
0,2000-01-03,0.936384,1.004464,0.907924,0.999442,535796800
1,2000-01-04,0.966518,0.987723,0.903460,0.915179,512377600
2,2000-01-05,0.926339,0.987165,0.919643,0.928571,778321600
3,2000-01-06,0.947545,0.955357,0.848214,0.848214,767972800
4,2000-01-07,0.861607,0.901786,0.852679,0.888393,460734400
...,...,...,...,...,...,...
5756,2022-11-16,149.130000,149.870000,147.290000,148.790000,64218266
5757,2022-11-17,146.430000,151.480000,146.150000,150.720000,80389400
5758,2022-11-18,152.305000,152.700000,149.970000,151.290000,74829573
5759,2022-11-21,150.160000,150.370000,147.715000,148.010000,58724070


In [20]:
hist_price = hist_price.rename(({'date': 'Date',
                                    'open': 'Open',
                                    'close': 'Close'}),
                              axis = 'columns')


In [21]:
hist_price['perc_change'] = ((hist_price['Close'] - hist_price['Open']) / 
                                hist_price['Open'])

In [22]:
hist_price

,Date,Open,high,low,Close,volume,perc_change
0,2000-01-03,0.936384,1.004464,0.907924,0.999442,535796800,0.067342
1,2000-01-04,0.966518,0.987723,0.903460,0.915179,512377600,-0.053118
2,2000-01-05,0.926339,0.987165,0.919643,0.928571,778321600,0.002410
3,2000-01-06,0.947545,0.955357,0.848214,0.848214,767972800,-0.104829
4,2000-01-07,0.861607,0.901786,0.852679,0.888393,460734400,0.031088
...,...,...,...,...,...,...,...
5756,2022-11-16,149.130000,149.870000,147.290000,148.790000,64218266,-0.002280
5757,2022-11-17,146.430000,151.480000,146.150000,150.720000,80389400,0.029297
5758,2022-11-18,152.305000,152.700000,149.970000,151.290000,74829573,-0.006664
5759,2022-11-21,150.160000,150.370000,147.715000,148.010000,58724070,-0.014318
